In [1]:
import numpy as np
import math
import sklearn # Only for downloading MNIST Dataset and Accuracy Metrics
import sklearn.metrics
from tensorflow.keras.utils import to_categorical  # Only for categorical one hot encoding
import random
from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import normalize

In [2]:
# %tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
# !pip3 install cupy-cuda101
# import cupy as np
# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.3.0-dev20200524


In [3]:
antiCategorical = lambda x: [np.argmax(i) for i in x]

linear = lambda x : x
sigmoid = lambda x: 1/(1+math.e**-x)
softmax = lambda x: math.e**x/(sum(math.e**x))
relu = lambda x: (x+(np.abs(x)))/2
elu = lambda x: np.array([i if i >=0  else 0 for i in x])
tanh = lambda x: np.tanh(x)
# softplus  = lambda x : np.vectorize(lambda x : x if x >= 50 else np.log(np.exp(x)+1))(x)#np.log(1 + np.exp(x))
softplus = lambda x : tf.nn.softplus(x)
leakyrelu = lambda x : tf.nn.leaky_relu(x / np.mean(x, axis=0), 0.01)


inverse_sigmoid = lambda x: np.log(x) - np.log(1-x)#1/(1+math.e**-x)
# inverse_softmax = lambda x: math.e**x/(sum(math.e**x))
inverse_linear = lambda x: x
# inverse_relu = lambda x: (x+(np.abs(x)))/2
# inverse_elu = lambda x: np.array([i if i >=0  else 0 for i in x])
inverse_tanh = lambda x: np.nan_to_num(0.5*(np.log( (1+x)/(1-x) )))#np.tanh(x)
# inverse_softplus = lambda x: np.log(np.exp(x) - 1.)
# inverse_softplus = lambda x: np.vectorize(lambda x : x if x >= 50 else (np.log(np.exp(x) - 1.)) if x > 0 else -30)(x)
def inverse_softplus(y):
  ys = np.maximum(y, np.finfo(np.float32).eps)
  return ys + np.log(-np.expm1(-ys))

inverse_tanh_continuous = lambda x : np.multiply(np.ones(x.shape), x >=5) + np.multiply(-np.ones(x.shape), x <-5) +  np.multiply(np.multiply(np.ones(x.shape), x < 5), x >= -5) * inverse_tanh(x)

inverse_leakyrelu = lambda x : np.multiply(x, x >=0) + np.multiply(x, x <0)*100

inverse_functions = {sigmoid:inverse_sigmoid, linear:inverse_linear, tanh:inverse_tanh, softplus:inverse_softplus, softmax:inverse_linear, leakyrelu:inverse_leakyrelu};

global idd
idd = 0

class Layer:
  def __init__(self):
    return None
  def forwardProp(self):
    return None
  def initWights(self):
    return None
  def backprop(self, Y):
    return None
  def update(self):
    return None

class Dense(Layer):
  def __init__(self, size, last_layer, activation=relu):
    global idd
    self.id = idd
    idd += 1
    self.size = size
    self.activation = activation
    self.weights = None
    self.sum = None
    self.last_layer = last_layer
    self.next_layer = None
    self.last_layer.next_layer = self

  def forwardProp(self):
    self.sums = np.dot(self.last_layer.forwardProp(), self.weights)
    self.outputs = self.activation(self.sums)
    return self.outputs

  def initWights(self):
    if self.next_layer is not None:
      self.weights = np.random.normal(0, scale=(1/float(math.sqrt(self.last_layer.size))), size=(self.last_layer.size, self.size))
      self.next_layer.initWights()

  def backprop(self, Y):
    self.Z = Y
#     self.Z, _ = tf.linalg.normalize(Y)
    Z = np.dot(inverse_functions[self.activation](self.Z), np.linalg.pinv(self.weights))
    lastOut = (self.last_layer.backprop(Z))
    self.weights = (np.dot(np.linalg.pinv(lastOut), inverse_functions[self.activation](self.Z)))
    return self.activation(np.dot(lastOut, self.weights))
    
  def update(self):
    return None

class Input(Layer):
  def __init__(self, size, activation=linear):
    global idd
    self.id = idd
    idd += 1
    self.size = size
    self.activation = activation
    self.outputs = np.zeros(size).astype(np.float64)
    self.sums = np.zeros(size).astype(np.float64)
    self.errors = None
    self.next_layer = None
  def forwardProp(self):
    # print("-->", self.outputs.shape)
    return self.activation(self.outputs)
  def initWights(self):
    if self.next_layer is not None:
      self.next_layer.initWights()
  def backprop(self, Y):
    return self.forwardProp()
  def update(self):
    return None

class NewNet:
  def __init__(self,  inputs, outputs):
    global idd
    idd = 0
    self.Input = inputs
    self.Output = outputs
  def forward(self, inputs):
    self.Input.outputs = inputs
    return self.Output.forwardProp()
  
  def fit(self, X, Y, verbose=True, validation_set = None, accuracy_metric=antiCategorical, epoch=1):
    maxShape = X.shape[1]

    self.Input.outputs = X
    self.Input.initWights()

    Xo, _ = tf.linalg.normalize(X)
    Yo = Y

    for i in range(epoch):
      X = Xo #+ noise
      noise =  np.random.rand(Y.shape[0], Y.shape[1]) / 10
      Y = Yo + noise
      self.Input.outputs = X

      self.Output.weights = (np.dot(np.linalg.pinv(self.Output.last_layer.forwardProp()), inverse_functions[self.Output.activation](Y)))
      Z = (np.dot(inverse_functions[self.Output.activation](Y), np.linalg.pinv(self.Output.weights)))
      Yy = (self.Output.last_layer.backprop(Z))

      self.Output.weights = (np.dot(np.linalg.pinv(Yy), inverse_functions[self.Output.activation](Y)))
      print("=>", np.mean(Z-Yy), "Loss: ", np.mean(Y - self.Output.activation(np.dot(Yy, self.Output.weights))))
      # print(self.weights.shape)
      if validation_set is not None:
        tx, ty = validation_set
        pred = self.predict(tx)
        print("Validation Score:", sklearn.metrics.accuracy_score(accuracy_metric(ty), accuracy_metric(pred))*100)
    return None
  
  def predict(self, X):
    return self.forward(X)

  def test(self, X, Y):
    preds = self.predict(X)
    print("Validation Score:", sklearn.metrics.accuracy_score(accuracy_metric(Y), accuracy_metric(pred))*100)

class MatrixNet:
  def __init__(self, activation=softplus):
    global idd
    idd = 0
    self.activation = activation
    self.weights = None
  def forward(self, inputs):
    print(inputs.shape, self.w0.shape)
    outputs = self.activation(np.dot(np.dot(inputs,self.w0), self.weights))
    return outputs
  
  def fit(self, X, Y, verbose=True, validation_set = None, accuracy_metric=antiCategorical):
    maxShape = X.shape[1]
    self.w0 = np.random.rand(maxShape, 784)
    self.weights = np.dot(np.linalg.pinv(np.dot(X, self.w0)), inverse_functions[self.activation](Y))
    # print(self.weights.shape)
    if validation_set is not None:
      tx, ty = validation_set
      pred = self.predict(tx)
      print("Validation Score:", sklearn.metrics.accuracy_score(accuracy_metric(ty), accuracy_metric(pred))*100)
    return None
  
  def predict(self, X):
    return self.forward(X)

In [4]:
from tensorflow.keras.datasets import fashion_mnist
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
cy_train = np.array(to_categorical(y_train))
cy_test = np.array(to_categorical(y_test))

cx_train, cx_test = np.array(x_train.reshape(-1, 784)/255.), np.array(x_test.reshape(-1, 784)/255.)
cx_train.shape

(60000, 784)

In [8]:
inp = Input(784, leakyrelu)
x = Dense(1024, inp, leakyrelu)
x = Dense(512, x, leakyrelu)
# x = Dense(512, x, leakyrelu)
x = Dense(512, x, leakyrelu)
# x = Dense(10, x, linear)
out = Dense(10, x, softmax)
net = NewNet(inp, out)

m = 60000
# net.fit(cx_train[:m], cy_train[:m] + noise, validation_set=(cx_train[:784], cy_train[:784]), verbose=True)
net.fit(cx_train[:m], cy_train[:m], validation_set=(cx_test, cy_test), verbose=True, epoch=1)

ResourceExhaustedError: OOM when allocating tensor with shape[60000,784] and type double on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:LeakyRelu]

In [6]:
preds = net.predict(cx_train)
print("Validation Score:", sklearn.metrics.accuracy_score(antiCategorical(cy_train), antiCategorical(preds))*100)

Validation Score: 85.40333333333334


In [6]:
weights1 = np.random.rand(784, 1024)
out = (np.dot(cx_train, weights1))
weights2 = np.random.rand(1024, 10)#np.dot(np.linalg.pinv(out), (cy_train + noise))
oldw2 = weights2
for i in range(0, 100):
  noise =  np.random.rand(cy_train.shape[0], cy_train.shape[1]) / 10
  preds = softmax(np.dot((np.dot(cx_test, weights1)), weights2))
  print("Epoch ", i, sklearn.metrics.accuracy_score(antiCategorical(cy_test), antiCategorical(preds))*100)

  Z = np.dot((cy_train+noise), tf.linalg.pinv(weights2))
  oldw1 = weights1
  weights1 = np.dot(tf.linalg.pinv(cx_train), (Z))
  W = np.dot(Z, tf.linalg.pinv(weights1))
  loss = cx_train - W
  print("Loss: ", np.mean(loss), np.mean(oldw1 - weights1), np.mean(oldw2 - weights2))
  out = (np.dot(cx_train, weights1))
  oldw2 = weights2
  weights2 = np.dot(tf.linalg.pinv(out), (cy_train + noise))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in power
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


Epoch  0 10.0
Loss:  0.2834924899170985 0.500301634531986 0.0
Epoch  1 80.9


KeyboardInterrupt: ignored

In [ ]:
U = np.dot(np.linalg.pinv(cx_train), W)

In [ ]:
weights2 = np.random.rand(1024, 10)

In [ ]:

weights1 = np.random.rand(784, 1024)

In [ ]:
out = (np.dot(cx_train, weights1))
noise =  np.random.rand(cy_train.shape[0], cy_train.shape[1]) / 100
weights2 = np.dot(np.linalg.pinv(out), (cy_train + noise))

In [ ]:
preds = (np.dot((np.dot(np.dot(cx_test, U), weights1)), weights2))

In [ ]:
sklearn.metrics.accuracy_score(antiCategorical(cy_test), antiCategorical(preds))*100

80.88

In [ ]:
weights2.shape

(1024, 10)

In [ ]:
Z = np.dot((cy_train+noise), np.linalg.pinv(weights2))

In [ ]:
N = (np.dot(cx_train, weights1))

In [ ]:
# noise =  np.random.rand(Z.shape[0], Z.shape[1]) / 100
weights1 = np.dot(np.linalg.pinv(cx_train), (Z))

In [ ]:
net = MatrixNet(softplus)
m = 60000
noise =  np.random.rand(m, 10) / 10
# net.fit(cx_train[:m], cy_train[:m] + noise, validation_set=(cx_train[:784], cy_train[:784]), verbose=True)
net.fit(cx_train[:m], cy_train[:m] + noise, validation_set=(cx_test, cy_test), verbose=True)

(10000, 784) (784, 1024)
Validation Score: 80.77


In [15]:
# Demo using tensorflow for Comparasion -->
import tensorflow as tf

inp = tf.keras.layers.Input(shape=(784,))
x = tf.keras.layers.LeakyReLU(0.01)(inp)
# x = tf.keras.activations.softplus(inp)
x = tf.keras.layers.Dense(1024, activation='linear')(x)
x = tf.keras.layers.LeakyReLU(0.01)(x)
x = tf.keras.layers.Dense(512, activation='linear')(x)
x = tf.keras.layers.LeakyReLU(0.01)(x)
# x = tf.keras.layers.Dense(30)(x)
out = tf.keras.layers.Dense(10, activation='softmax')(inp)

model = tf.keras.models.Model(inputs=inp, outputs=out)
model.compile(optimizer='Adam', loss="mse", metrics=["mae", "accuracy"])
model.fit(cx_train, cy_train, batch_size=40, validation_data=(cx_test, cy_test), epochs=10, shuffle=True)

Epoch 1/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0302 - mae: 0.0680 - accuracy: 0.7960 - val_loss: 0.0258 - val_mae: 0.0543 - val_accuracy: 0.8261
Epoch 2/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0233 - mae: 0.0505 - accuracy: 0.8440 - val_loss: 0.0239 - val_mae: 0.0500 - val_accuracy: 0.8378
Epoch 3/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0219 - mae: 0.0463 - accuracy: 0.8532 - val_loss: 0.0233 - val_mae: 0.0468 - val_accuracy: 0.8386
Epoch 4/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0211 - mae: 0.0440 - accuracy: 0.8581 - val_loss: 0.0227 - val_mae: 0.0454 - val_accuracy: 0.8454
Epoch 5/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0207 - mae: 0.0426 - accuracy: 0.8608 - val_loss: 0.0223 - val_mae: 0.0438 - val_accuracy: 0.8483
Epoch 6/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0204 - mae: 0.0414 - accuracy: 0.8632 - va